In [1]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import joblib
import random
from sklearn.feature_selection import VarianceThreshold

In [6]:
raw_train = pd.read_csv('all_extracted_features_and_decision_values_train_100node.csv')
raw_test = pd.read_csv('all_extracted_features_and_decision_values_test_100node.csv')
raw_train.head()

,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,dv0,dv1,isdnumber,Early recur
0,0.157632,0.061002,0.109447,-0.123675,-0.037835,0.031847,0.155704,-0.134332,0.007117,0.057411,...,-0.147874,-0.100417,-0.081311,-0.046580,0.205537,-0.055564,0.530573,0.469427,1097,1
1,0.175911,0.152456,0.175580,-0.129992,-0.013403,0.037462,0.154017,-0.135216,0.034047,0.118865,...,-0.136819,-0.106656,-0.090762,-0.046667,0.258936,-0.081796,0.483123,0.516877,1097,1
2,0.158855,0.063547,0.111752,-0.123899,-0.038091,0.031881,0.156181,-0.134110,0.008132,0.058993,...,-0.148154,-0.101476,-0.081419,-0.046723,0.206916,-0.056505,0.529299,0.470701,1097,1
3,0.158853,0.067147,0.114853,-0.125452,-0.037032,0.031887,0.156542,-0.135139,0.009093,0.061856,...,-0.147961,-0.102122,-0.082515,-0.046519,0.209061,-0.057097,0.527139,0.472861,1097,1
4,0.159575,0.069426,0.117512,-0.124833,-0.036489,0.032769,0.156704,-0.135534,0.010537,0.064531,...,-0.148351,-0.102107,-0.082187,-0.046994,0.210010,-0.058433,0.525699,0.474301,497,1


要先用each_add2_OPpath2_test.csv去補齊all_extracted_features_and_decision_values_test_100node.csv缺的isdnumber跟Early recur

In [14]:
import pandas as pd

# 載入CSV檔案
# file_path = '/path/to/your/csv/file.csv'  # 請將這裡的路徑替換成您的CSV檔案路徑
# data = pd.read_csv(file_path)

# 定義一個函數來根據指定條件選擇行
def select_row(group):
    if group['Early recur'].eq(1).any():
        # 如果'Early recur'至少有一個是1，則選擇'dv1'值最大的行
        return group.loc[group['dv1'].idxmax()]
    else:
        # 如果'Early recur'全部是0，則選擇'dv0'值最小的行
        return group.loc[group['dv0'].idxmin()]

selected_data_train = raw_train.groupby('isdnumber', group_keys=False).apply(select_row)

# 重置索引以清理數據框
selected_data_train.reset_index(drop=True, inplace=True)

# 顯示處理後的數據的前幾行
print(len(selected_data_train))
selected_data_train.to_csv('process_train.csv',index=False)
# 對每個'isdnumber'組應用這個函數
selected_data_test = raw_test.groupby('isdnumber', group_keys=False).apply(select_row)

# 重置索引以清理數據框
selected_data_test.reset_index(drop=True, inplace=True)

# 顯示處理後的數據的前幾行
print(len(selected_data_test))
selected_data_test.to_csv('process_test.csv',index=False)


410
178


In [16]:
train = pd.read_csv('./process_train.csv')
test = pd.read_csv('./process_test.csv')


In [17]:
#製作allNAP_clinical_test_std_v5時
columns_to_drop = ['isdnumber','Early recur']
# ylabel = ['ISD number','Early recur']
X_train = train.drop(columns_to_drop,axis=1)
y_train = train[['isdnumber','Early recur']]
X_test = test.drop(columns_to_drop,axis=1)
y_test = test[['isdnumber','Early recur']]
scaler = StandardScaler()
scaler.fit(X_train)
scaled_X_train = pd.DataFrame(scaler.transform(X_train), columns=X_train.columns)
scaled_X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)
display(scaled_X_train.tail())
display(y_train.tail())

scaled_data = pd.concat([scaled_X_train, y_train], axis=1)
scaled_data_test = pd.concat([scaled_X_test, y_test], axis=1)
scaled_data.to_csv('./process_train_std.csv',index=False)
scaled_data_test.to_csv('./process_test_std.csv',index=False)
joblib.dump(scaler, './process_train_scaler.bin')

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,dv0,dv1
405,0.566764,0.492245,0.533487,-0.253387,0.471455,0.573040,0.684916,-0.778152,0.489480,0.535984,...,0.509729,0.514818,0.414510,-0.626920,-0.486878,0.198832,0.458981,-0.527142,-0.522671,0.522671
406,-0.992527,-1.025422,-1.023694,0.912213,-1.025544,-1.074872,-0.734132,0.967397,-1.012898,-1.023471,...,-1.015136,-1.021860,-1.034408,0.916656,0.934095,-0.321036,-1.011021,1.017082,1.036562,-1.036561
407,1.243657,1.279568,1.257449,-1.403821,1.288701,1.342489,0.997555,-1.009119,1.302060,1.287617,...,1.298904,1.271518,1.029228,-1.442961,-1.313334,0.527546,1.295395,-1.211470,-1.295784,1.295784
408,1.682107,1.743369,1.757028,-1.772074,1.718086,1.521927,1.274387,-1.498530,1.764429,1.725777,...,1.773637,1.752734,2.016468,-1.672257,-1.664067,0.904282,1.770271,-1.775369,-1.740060,1.740061
409,1.682850,1.700811,1.687494,-1.633070,1.739968,1.781356,1.919993,-1.325901,1.643020,1.698623,...,1.689510,1.691937,1.808233,-1.662709,-1.549470,1.095073,1.739645,-1.660774,-1.686589,1.686590


,isdnumber,Early recur
405,1222.0,1.0
406,1225.0,0.0
407,1229.0,1.0
408,1230.0,1.0
409,1234.0,0.0


['./process_train_scaler.bin']

將正規化後的資料合併回去

In [51]:
train_std = pd.read_csv('./process_train_std.csv')
test_std = pd.read_csv('./process_test_std.csv')
print(train_std.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '94', '95', '96', '97', '98', '99', 'dv0', 'dv1', 'isdnumber',
       'Early recur'],
      dtype='object', length=104)


In [52]:
threed_train = pd.read_csv('../OP_clinical+radiomics+3d survival/data/cli_radi_morphology_3d_train_nphase_id.csv')
threed_test = pd.read_csv('../OP_clinical+radiomics+3d survival/data/cli_radi_morphology_3d_test_nphase_id.csv')
print(len(threed_train.columns))

131


In [49]:
print(threed_test.columns)

Index(['Tumor size', 'lnAST', 'BCLC', 'Tumor number', 'ALBIgrade',
       'class_Histologic grade', 'class_AFP', 'Ishak', 'Steatosis grade', 'K',
       ...
       'n_aspect_ratio_3', 'n_max_radius', 'n_abb_volume',
       'n_sphericity_ellipticity', 'n_defect_ratio', 'n_gauss_curv_mean',
       'n_mean_curv_mean', 'ISD number', 'Early recur', 'Early RFS'],
      dtype='object', length=131)


In [ ]:
import pandas as pd

train_std.rename(columns={'isdnumber': 'ISDnumber'}, inplace=True)

# 根據ISD number合併兩個數據框
merged_data = pd.merge(threed_train, train_std, on='ISDnumber', how='inner')
# print(merged_data.columns)
# 刪除重複的ISD number和Early recur欄位
# 假設重複的欄位在合併後分別命名為 'ISD number_x', 'ISD number_y', 'Early recur_x', 'Early recur_y'
# 我們需要刪除其中的一組，這裡以 '_y' 結尾的欄位為例
merged_data.drop(columns=[ 'Early recur_y'], inplace=True)

# 重新命名剩餘的欄位以去除 '_x'
merged_data.rename(columns={ 'Early recur_x': 'Early recur'}, inplace=True)


# 假設 merged_data 是您已經合併並處理過的數據框

# 選擇要移動的列
columns_to_move = ['ISDnumber', 'Early recur', 'Early RFS']

# 將這些列從原數據框中刪除，並暫存到一個新的DataFrame
temp_df = merged_data[columns_to_move]

# 從原數據框中刪除這些列
merged_data.drop(columns=columns_to_move, inplace=True)

# 將暫存的列添加到原數據框的末尾
merged_data = pd.concat([merged_data, temp_df], axis=1)

# 檢視修改後的數據框
print(len(merged_data))
merged_data.to_csv('./cli_radi_morphology_3d_DL_nphase_train_id__std.csv',index=False)

In [53]:
import pandas as pd

test_std.rename(columns={'isdnumber': 'ISD number'}, inplace=True)

# 根據ISD  number合併兩個數據框
merged_data = pd.merge(threed_test, test_std, on='ISD number', how='inner')
# print(merged_data.columns)
# 刪除重複的ISD  number和Early recur欄位
# 假設重複的欄位在合併後分別命名為 'ISD  number_x', 'ISD  number_y', 'Early recur_x', 'Early recur_y'
# 我們需要刪除其中的一組，這裡以 '_y' 結尾的欄位為例
merged_data.drop(columns=[ 'Early recur_y'], inplace=True)

# 重新命名剩餘的欄位以去除 '_x'
merged_data.rename(columns={ 'Early recur_x': 'Early recur'}, inplace=True)


# 假設 merged_data 是您已經合併並處理過的數據框

# 選擇要移動的列
columns_to_move = ['ISD number', 'Early recur', 'Early RFS']

# 將這些列從原數據框中刪除，並暫存到一個新的DataFrame
temp_df = merged_data[columns_to_move]

# 從原數據框中刪除這些列
merged_data.drop(columns=columns_to_move, inplace=True)

# 將暫存的列添加到原數據框的末尾
merged_data = pd.concat([merged_data, temp_df], axis=1)

# 檢視修改後的數據框
print(len(merged_data))
merged_data.to_csv('./cli_radi_morphology_3d_DL_nphase_test_id__std.csv',index=False)

178
